<a href="https://colab.research.google.com/github/zainali78690/IBM-Course/blob/main/Dummy_Stable_ID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import random
import numpy as np


# Things to do for this to happen
  # import full table into class to compare to
  # In this case new_org_entries is kb_org_table, just need to assign IDs from old and generate for new


# Pre Steps - DONE
  # Import initial saved table with entity id, and identifiers - table is kb_org_table
  # Save this file to a CSV to call later

# Processing Steps - DONE
  # use org_subset table and merge with imported table to find if its duplicated
  # merge on each identifier to see (if thats an issue)
  # If duplicated draw the KPMGID from the table and append to table

# Post steps 
  # Saved file will be flattened kb_org_table similiar to org_subset
  # Change names where outputted table from final function is kb_org_table, new_entities_df = org_subset
  # Where to apply in generate_kpmg_id - create 2 seperate functions and then function to call in create IDs
  # Create Home/Dir for saved file
  # Ensure final dataframe has duplicated ids run on it - 'find_and_replace_duplicated_uuids_across_dataframes' function

In [ ]:
data = {'KPMGID':['abc123def456', 'hij123klm456', 'nop123qrs456','TEST1','TEST2'], 'Identifier':['', '', 'US1234567893','TEST1','TEST2'], 
        'Identifier2':['US1234567896', 'US1234567897', '','TEST1','TEST2'],'Identifier3':['', '', 'US1234567894','TEST1','TEST2'],
        'Identifier4':['', '', '','TEST1','US1234567899'], 'Identifier5':['', '', '','TEST1','US1234567899']}
df = pd.DataFrame(data)
df.to_csv('entity_db.csv')
df

,KPMGID,Identifier,Identifier2,Identifier3,Identifier4,Identifier5
0,abc123def456,,US1234567896,,,
1,hij123klm456,,US1234567897,,,
2,nop123qrs456,US1234567893,,US1234567894,,
3,TEST1,TEST1,TEST1,TEST1,TEST1,TEST1
4,TEST2,TEST2,TEST2,TEST2,US1234567899,US1234567899


In [ ]:
new_entities = {'Identifier':['', '', 'US1234567893', '123ufrmfkrnvjr','TEST2'], 'Identifier2':['US1234567896', 'US1234567897', '', 'uhfuwrf89ef','TEST2'], 'Identifier3':['', '', 'US1234567894', '','TEST2' ],
                'Identifier4':['', '', '','','US1234567899'], 'Identifier5':['', '', '','','US1234567899']}
new_entities_df = pd.DataFrame(new_entities)
new_entities_df = new_entities_df.replace(r'^\s*$', np.nan, regex=True)
new_entities_df

,Identifier,Identifier2,Identifier3,Identifier4,Identifier5
0,NaN,US1234567896,NaN,NaN,NaN
1,NaN,US1234567897,NaN,NaN,NaN
2,US1234567893,NaN,US1234567894,NaN,NaN
3,123ufrmfkrnvjr,uhfuwrf89ef,NaN,NaN,NaN
4,TEST2,TEST2,TEST2,US1234567899,US1234567899


In [ ]:
identifier_list = ['Identifier','Identifier2','Identifier3','Identifier4','Identifier5']

In [ ]:
# The objective of this is to search through the imported db to see if the new entites are already in the imported entity db - using merge
def entity_id_search(entity_data, identifier_list, dataframe_type):
  # if statement for data load
  if dataframe_type == 'entity':
    #Load dataframe
    imported_data = pd.read_csv("entity_db.csv")
    imported_data = imported_data.iloc[: , 1:]
    imported_data = imported_data.astype(str).replace('nan',np.nan)
    imported_data = imported_data.fillna('')

  entity_data = entity_data.fillna('')

  #Create empty dataframe to append rows
  merged_df = pd.DataFrame()
  for identifier in identifier_list:
    keys = [identifier]
    search_results = imported_data.merge(entity_data[keys], on=keys)
    merged_df = pd.concat([merged_df,search_results])

  #Remove Duplicate ID entries - add  
  drop_duplicates = merged_df.drop_duplicates(keep = 'first')
  

  return drop_duplicates


In [ ]:
entity_id_search(new_entities_df, identifier_list, 'entity')

,KPMGID,Identifier,Identifier2,Identifier3,Identifier4,Identifier5
0,abc123def456,,US1234567896,,,
2,hij123klm456,,US1234567897,,,
4,nop123qrs456,US1234567893,,US1234567894,,
5,TEST2,TEST2,TEST2,TEST2,US1234567899,US1234567899


In [ ]:
# Generate KPMG_ID
def generate_uuid() -> str:
    """Generates a unique 12 character id.
    Returns:
        str: a 12 character unique identifier
    """
    return random.randint(100000000000,999999999999)

In [ ]:
identifier_tuple = tuple(identifier_list)
identifier_tuple 

('Identifier', 'Identifier2', 'Identifier3', 'Identifier4', 'Identifier5')

In [ ]:
list_test = ['Identifier', 'Identifier2', 'Identifier3']
list_test  = list_test + ['Identifier4'] + ['identifier5']
list_test


['Identifier', 'Identifier2', 'Identifier3', 'Identifier4', 'identifier5']

In [ ]:
# Create ID for non matched data in new data for 

matched = entity_id_search(new_entities_df, identifier_list, 'entity')
dataframe_type = 'entity'
  
#if statement to assign KPMGID name
if dataframe_type == 'entity':
  dataframe_id = 'KPMGID'
else:
  dataframe_id = 'KPMG_INS'

# IF STATEMET DEPENDING ON dataframe_str
# Make table able for merge (ensure both dfs have empty values not NAN)
matched_no_id = matched.fillna('')
matched_no_id = matched_no_id.drop([dataframe_id], axis = 1)

new_entities_df = new_entities_df.fillna('')

df_left = new_entities_df.merge(matched_no_id, on = identifier_tuple, how = 'left', indicator = True)
df_left
df_outer = df_left[df_left['_merge'] == 'left_only']
df_outer = df_outer.drop('_merge', axis = 1)
df_outer[dataframe_id] = df_outer.apply(lambda x: generate_uuid(), axis=1)
df_outer = df_outer[ [dataframe_id] + [ col for col in df_outer.columns if col != dataframe_id ] ]

#Combine 2 tables for final output - ask Yang if NAN or '' needed
combined_df = pd.concat([matched, df_outer], ignore_index=True, sort=False)
combined_df = combined_df.fillna('')
combined_df

,KPMGID,Identifier,Identifier2,Identifier3,Identifier4,Identifier5
0,abc123def456,,US1234567896,,,
1,hij123klm456,,US1234567897,,,
2,nop123qrs456,US1234567893,,US1234567894,,
3,TEST2,TEST2,TEST2,TEST2,US1234567899,US1234567899
4,514729193262,123ufrmfkrnvjr,uhfuwrf89ef,,,


In [ ]:
def entity_id_search(dataframe, identifier_list, dataframe_type):
    #Load dataframe and pre-process for merge
    if dataframe_type == 'entity':
        imported_data = pd.read_csv("entity_instrument_management/files/kb_org_table.csv")
        imported_data = imported_data.iloc[: , 1:]
        imported_data = imported_data.astype(str).replace('nan',np.nan)
        imported_data = imported_data.fillna('')
    else:
        imported_data = pd.read_csv("entity_instrument_management/files/kb_ins_table.csv")
        imported_data = imported_data.iloc[: , 1:]
        imported_data = imported_data.astype(str).replace('nan',np.nan)
        imported_data = imported_data.fillna('')

    dataframe = dataframe.fillna('')

    #Create empty dataframe to append rows
    merged_df = pd.DataFrame()
    
    #Loop for merge
    for identifier in identifier_list:
        keys = [identifier]
        search_results = imported_data.merge(dataframe[keys], on=keys)
        merged_df = pd.concat([merged_df,search_results])

    #Remove Duplicate ID entries from merge
    drop_duplicates = merged_df.drop_duplicates(keep = 'first')

    return drop_duplicates


In [ ]:
def generate_id_new(dataframe, identifier_list, dataframe_type):
    # Create ID for matched data in new file with saved file
    matched = entity_id_search(dataframe, identifier_list, dataframe_type)
    
    if dataframe_type == 'instrument':
        identifier_tuple = tuple(identifier_list)
    else:
        identifier_list = identifier_list + ['bw_entity'] + ['sector']
        identifier_tuple = tuple(identifier_list)

    # if statement to assign KPMGID name
    if dataframe_type == 'entity':
        dataframe_id = 'kpmg_entity_id'
    else:
        dataframe_id = 'kpmg_instrument_id'

    # IF STATEMET DEPENDING ON dataframe_str - (why needed)
    # Make table able for merge (ensure both dfs have empty values not NAN)
    matched_no_id = matched.fillna('')
    matched_no_id = matched_no_id.drop([dataframe_id], axis = 1)

    dataframe = dataframe.fillna('')

    # Create ID for non matched data in new data for 
    df_left = dataframe.merge(matched_no_id, on = identifier_tuple, how = 'left', indicator = True)
    df_outer = df_left[df_left['_merge'] == 'left_only']
    df_outer = df_outer.drop('_merge', axis = 1)
    df_outer[dataframe_id] = df_outer.apply(lambda x: generate_uuid(), axis=1)
    df_outer = df_outer[ [dataframe_id] + [ col for col in df_outer.columns if col != dataframe_id ] ]

    #Combine 2 tables for final output - ask Yang if NAN or '' needed
    combined_df = pd.concat([matched, df_outer], ignore_index=True, sort=False)
    combined_df = combined_df.fillna('')

    #Overwrite csv
    if dataframe_type == 'entity':
        combined_df.to_csv("entity_instrument_management/files/kb_org_table.csv",mode='w+')
    else:
        combined_df.to_csv("entity_instrument_management/files/kb_ins_table.csv", mode = 'w+')

        
    #Rename Column to 'KPMG_uuid'
    if dataframe_type == 'entity':
        combined_df_rename = combined_df.rename(columns = {'kpmg_entity_id':'kpmg_uuid'})
    else:
        combined_df_rename = combined_df.rename(columns = {'kpmg_instrument_id':'kpmg_uuid'})


    return combined_df_rename


In [ ]:
df_outer

,KPMGID,Identifier,Identifier2,Identifier3,Identifier4,Identifier5
3,513866078113,123ufrmfkrnvjr,uhfuwrf89ef,,,


In [ ]:
def generate_id_new(entity_data, identifier_list, dataframe_type):
  # Create ID for matched data in new file with saved file
  matched = entity_id_search(entity_data, identifier_list)

  # TURN LIST INTO TUPLE
  identifier_tuple = tuple(identifier_list)
   
  # if statement to assign KPMGID name
  if dataframe_type == 'entity':
    dataframe_id = 'KPMGID'
  else:
    dataframe_id = 'KPMG_INS'

  # IF STATEMET DEPENDING ON dataframe_str
  # Make table able for merge (ensure both dfs have empty values not NAN)
  matched_no_id = matched.fillna('')
  matched_no_id = matched_no_id.drop([dataframe_id], axis = 1)

  entity_data = entity_data.fillna('')

  # Create ID for non matched data in new data for 
  df_left = entity_data.merge(matched_no_id, on = identifier_tuple, how = 'left', indicator = True)
  df_outer = df_left[df_left['_merge'] == 'left_only']
  df_outer = df_outer.drop('_merge', axis = 1)
  df_outer[dataframe_id] = df_outer.apply(lambda x: generate_uuid(), axis=1)
  df_outer = df_outer[ [dataframe_id] + [ col for col in df_outer.columns if col != dataframe_id ] ]

  #Combine 2 tables for final output - ask Yang if NAN or '' needed
  combined_df = pd.concat([matched, df_outer], ignore_index=True, sort=False)
  combined_df = combined_df.fillna('')

  #Overwrite csv
  combined_df.to_csv('entity_db.csv',mode='w+' )

  #Add NaN values if needed


  return combined_df

In [ ]:
generate_id_new(new_entities_df, identifier_list, 'entity')

,KPMGID,Identifier,Identifier2,Identifier3,Identifier4,Identifier5
0,abc123def456,,US1234567896,,,
1,hij123klm456,,US1234567897,,,
2,nop123qrs456,US1234567893,,US1234567894,,
3,TEST2,TEST2,TEST2,TEST2,US1234567899,US1234567899
4,644495824515,123ufrmfkrnvjr,uhfuwrf89ef,,,


In [ ]:
def recover_ids(self, table_subset, dataframe_type):

  if dataframe_type not in ["entity", "instrument"]:
    logger.error("dataframe_type must be either entity or instrument")
    raise ValueError

  if dataframe_type == 'entity':
    new_matches = generate_id_new(table_subset, org_table_columns_primary_key[0:-1])
  else:
    new_matches = generate_id_new(table_subset, ins_table_columns_primary_key)

  dataframe_with_dedup_id = replace_duplicated_uuids(
      new_matches, list_of_ids=self.previous_ids_used
  )

  #if length of ID <13 at prefix depending on if entity or instrument
  


